In [ ]:
%%writefile requirements.txt
lime
keras
pandas
tensorflow

In [ ]:
!pip install -r requirements.txt


In [ ]:
pip install scikeras[tensorflow]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as sk
#from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dati/dataset_taggato.csv', sep = ';')

#display(df)

In [ ]:
x = df.iloc[: , 1:16]
cols = x.columns
x[cols] = x[cols].replace(',','.', regex = True)
x[cols] = x[cols].apply(pd.to_numeric,downcast='float', errors='raise')
x = x.astype('float64')

scaler = StandardScaler()

x = pd.DataFrame(scaler.fit_transform(x.values), columns=x.columns, index=x.index)

#display(x)

#print(x.dtypes)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler2 = MinMaxScaler()
y = df.iloc[: , 28:30]
cols = y.columns
y[cols] = y[cols].replace(',','.', regex = True)

y[cols] = y[cols].apply(pd.to_numeric,downcast='float', errors='raise')

y = pd.DataFrame(scaler2.fit_transform(y.values), columns=y.columns, index=y.index)

#display(y.head(100))
#print(y.dtypes)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
import tensorflow as tf
from tensorflow import keras
model = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/nn_model")

In [ ]:
predictions = model.predict(x_test)

In [ ]:
l = 99
print(predictions[l])
print(y_test.iloc[l])

In [ ]:
def get_predict_fn(idx):
  def predict_fn(x_test):
    return model.predict(x_test)[:, idx]
  return predict_fn

#LIME

In [ ]:
import lime
import lime.lime_tabular

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(x_train.values, feature_names=list(x_train.columns), verbose=False, mode='regression', sample_around_instance=True, kernel_width=0.4)

In [ ]:
# from sklearn.linear_model import ElasticNet
# from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import SGDRegressor
# from sklearn.linear_model import LogisticRegression

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(x_train.values, feature_names=list(x_train.columns), verbose=False, mode='regression', sample_around_instance=True, kernel_width=0.56)
exp1 = explainer.explain_instance(x_test.iloc[l], predict_fn = get_predict_fn(0), num_features=15, num_samples=100, model_regressor = LinearRegression())
exp1.show_in_notebook()

In [ ]:
import time
start_time = time.time()

scores = []
for i in np.arange(0.01, 1, 0.05): #500
  print('kernel ='+ str(i))
  print("--- %s seconds ---" % (time.time() - start_time))

  explainer = lime.lime_tabular.LimeTabularExplainer(x_train.values, feature_names=list(x_train.columns), verbose=False, mode='regression', sample_around_instance=True, kernel_width=i)
  for f in range (50, 1000, 50):
    v = []
    a = []
    for h in range(5):
      exp1 = explainer.explain_instance(x_test.iloc[l], predict_fn = get_predict_fn(0), num_features=15, num_samples=f, model_regressor = LinearRegression())
      exp2 = explainer.explain_instance(x_test.iloc[l], predict_fn = get_predict_fn(1), num_features=15, num_samples=f, model_regressor =  LinearRegression())
      v.append(exp1.score)
      a.append(exp2.score)
      
    v = np.asarray(v)
    a = np.asarray(a)
    media_v = np.mean(v, dtype=np.float64)
    media_a = np.mean(a, dtype=np.float64)      
    print("Numero di Vicini: "+str(f))

    print('['+str(media_v)+', '+str(media_a)+']')
    score = [media_v, media_a]
    scores.append(score)

In [ ]:
scores = np.asarray(scores)
score_indexes = []
score_values = []
print (scores.shape)
for inti in range(scores.shape[0]):
    if scores[inti, 0] >=0.1 or scores[inti, 1] >=0.1:
        score_indexes.append(inti)
        score_values.append(scores[inti])
        

In [ ]:
savetxt('/content/drive/MyDrive/Colab Notebooks/data_indexes.csv', scores, delimiter=',')

In [ ]:
def feature_finder(s):
  char = "" 
  for x in s: 
    if x.isalpha(): 
        char = "".join([char, x])
  return char

In [ ]:
features = x_train.columns.tolist()
dictionary_v_p = { feature : 0 for feature in features }
dictionary_v_n = { feature : 0 for feature in features }
dictionary_a_p = { feature : 0 for feature in features }
dictionary_a_n = { feature : 0 for feature in features }


In [ ]:
val = []
ar = []
kw = 0.56
f = 150
explainer = lime.lime_tabular.LimeTabularExplainer(x_train.values, feature_names=list(x_train.columns), verbose=False, mode='regression', sample_around_instance=True, kernel_width=kw)
for i in range(1000):
  #print(str(i)+" di 1000")
  exp1 = explainer.explain_instance(x_test.iloc[l], predict_fn = get_predict_fn(0), num_features=15, num_samples=f, model_regressor = LinearRegression())
  exp2 = explainer.explain_instance(x_test.iloc[l], predict_fn = get_predict_fn(1), num_features=15, num_samples=f, model_regressor =  LinearRegression())
  li1 = exp1.as_list()
  li2 = exp2.as_list()
  val.append(exp1.score)
  ar.append(exp2.score)
  
  for lun in range(len(li1)):
    valore = li1[lun][1]
    feature = feature_finder(li1[lun][0])
    if feature == "polarityvalue":
      feature = "polarity_value" 
    if valore >0:
      dictionary_v_p[feature] = dictionary_v_p[feature]+valore
    else:
      dictionary_v_n[feature] = dictionary_v_n[feature]+valore


  for lun in range(len(li2)):
    valore = li2[lun][1]
    feature = feature_finder(li2[lun][0])
    if feature == "polarityvalue":
      feature = "polarity_value" 
    if valore >0:
      dictionary_a_p[feature] = dictionary_a_p[feature]+valore
    else:
      dictionary_a_n[feature] = dictionary_a_n[feature]+valore

print(dictionary_v_p)
print(dictionary_v_n)
print(dictionary_a_p)
print(dictionary_a_n)

In [ ]:
Valence = []
Arousal = []
for i in range(len(val)):
  Valence.append([val[i]])
  Arousal.append([ar[i]])

val = np.array(Valence)
ar = np.array(Arousal)

print(np.mean(val))
print(np.mean(ar))

In [ ]:
import json 
with open("/content/drive/MyDrive/Colab Notebooks/LIME_dictionary_v_n.json", "w") as outfile:
    json.dump(dictionary_v_n, outfile)
      
with open("/content/drive/MyDrive/Colab Notebooks/LIME_dictionary_v_n.json", "w") as outfile:
    json.dump(dictionary_v_n, outfile)

with open("/content/drive/MyDrive/Colab Notebooks/LIME_dictionary_v_p.json", "w") as outfile:
    json.dump(dictionary_v_p, outfile)

with open("/content/drive/MyDrive/Colab Notebooks/LIME_dictionary_a_p.json", "w") as outfile:
    json.dump(dictionary_a_p, outfile)

with open("/content/drive/MyDrive/Colab Notebooks/LIME_dictionary_a_n.json", "w") as outfile:
    json.dump(dictionary_a_n, outfile)